In [1]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
def get_batch(X_train, index):
    i = index
    result = []
    for val in X_train:
        result.append(val[i])
        
    return np.array(result)

batch = get_batch(df['EMBED'], 385)
preds = clf.predict(batch)


classification_report(
  df['LABEL'], 
    preds,
  output_dict=True)



    

In [3]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from os import walk
from tqdm import tqdm

In [4]:
"""
    site sizes:
    {'02': 1062, '03': 1087, '04': 1486, 
     '05': 2576, '06': 1804, '08': 1274, 
     '10': 1584, '11': 354, '14': 336, 
     '17': 70, '20': 439}

"""
mypath = '/scratch/users/benson97/embeddings_rif/'
# resultspath = mypath + 'results/' + 'rif_predictions_200_400_full_embedding.npy'
df = pd.read_csv('/scratch/users/benson97/' + 'rif_labels.csv')

# modify the number of sites as 
# we upload more sites to Sherlock...
sites = [
    '02',
    '03',
    '04',
    '05',
    '06',
    '08',
    '10',
    '11',
    '14',
    '17',
    '20'
]

site_dfs = []
for s in sites:
    site_dfs.append(
        df[df.UNIQUEID.str.contains('site.' + s)].reset_index(drop=True)
    )

df = pd.concat(site_dfs).reset_index(drop=True)

def get_site(uniqueid):
    return uniqueid.split('.')[1]

X = []
for uniqueid in tqdm(df['UNIQUEID']):
    if not os.path.exists(mypath + 'site_' + get_site(uniqueid) + '/' + uniqueid + '.npy'):
        print('Oh no! ' + uniqueid + ' is missing!')
    else:
#         X.append(
#             np.load(mypath + 'site_' + get_site(uniqueid) + '/' + uniqueid + '.npy').flatten()
#         )
        
       X.append(
            np.load(mypath + 'site_' + get_site(uniqueid) + '/' + uniqueid + '.npy')
        )
        
# X = pd.DataFrame({'EMBED':X})

# y = [
#     0 if label == 'S' else 1 for label in df['RIF_BINARY_PHENOTYPE']
# ]
# y = pd.DataFrame({'LABEL':y})

# df = pd.concat([df, X, y], axis=1)


100%|██████████| 12072/12072 [07:50<00:00, 25.65it/s]


In [5]:
def cosine_similarity(vec1, vec2):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    
    return dot_product / (norm_vec1 * norm_vec2)

def calculate_sim(embed):
    result = []
    for i in range(len(embed)):
        for j in range(i + 1, len(embed)):
            result.append(cosine_similarity(embed[i], embed[j]))
    return result

In [6]:
def calculate_statistics(results):
    return {
        'mean': np.mean(results),
        'std': np.std(results)
    }

In [9]:
X = pd.DataFrame({'EMBED':X})

In [10]:
samples = X.sample(n=1000)
samples = samples.to_list()

AttributeError: 'DataFrame' object has no attribute 'to_list'

In [ ]:
statistics = []
for sample in tqdm(samples['EMBED']):
    statistics.append(
        calculate_statistics(calculate_sim(sample))
    )
np.save('sample_n_1000_statistics.npy', statistics)

 15%|█▌        | 150/1000 [06:24<36:25,  2.57s/it]

In [ ]:
np.save('sample_n_1000_statistics.npy', statistics)

In [ ]:
sites = []
for id in df['UNIQUEID']:
    sites.append(get_site(id))
sites = pd.DataFrame({'SITE':sites})

df = pd.concat([df, sites], axis=1)

In [ ]:
df_2 = df[df.SITE == '02']

In [ ]:
i = 385
def get_batch(X_train, index):
    i = index
    result = []
    for val in X_train:
        result.append(val[i])
    return np.array(result)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_2['EMBED'],
                                                   df_2['LABEL'],
                                                   test_size=0.2,
                                                   random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_test, 
                                               y_test, 
                                               test_size=0.5, 
                                               random_state=42)

X_train_i = get_batch(X_train, i)
clf_2 = LogisticRegression(
    max_iter=1000,
    n_jobs=-1,
    C=10
)

clf_2.fit(X_train_i, y_train.to_list())

X_test_i = get_batch(X_test, i)
test_pred_2 = clf.predict_proba(X_test_i)

two_y_test = y_test.to_list()
two_y_preds = test_pred_2

# fpr, tpr, thresholds = metrics.roc_curve(y_test.to_list(), test_pred_ALL[:,1])
# roc_auc = metrics.auc(fpr, tpr)
# display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,
#                                   estimator_name='All sites')

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(two_y_test, two_y_preds[:,1])
auc = metrics.roc_auc_score(two_y_test, two_y_preds[:,1])
plt.plot(fpr, tpr, label='%s ROC (area = %0.2f)' % ('model', auc))

In [ ]:
from sklearn import metrics

# X_test_i = get_batch(X_test, i)
# test_pred_ALL = clf.predict_proba(X_test_i)

# all_y_test = y_test.to_list()
# all_y_preds = test_pred_ALL
fpr, tpr, thresholds = metrics.roc_curve(two_y_test, two_y_preds[:,1])
auc = metrics.roc_auc_score(two_y_test, two_y_preds[:,1])
plt.plot(fpr, tpr, label='%s ROC (area = %0.2f)' % ('Site 2', auc))

fpr, tpr, thresholds = metrics.roc_curve(all_y_test, all_y_preds[:,1])
auc = metrics.roc_auc_score(all_y_test, all_y_preds[:,1])
plt.plot(fpr, tpr, label='%s ROC (area = %0.2f)' % ('All sites', auc))

plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('1 - Specificity (False Positive Rate)')
plt.ylabel('Sensitivity (True Positive Rate)')
# plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
# plt.show()   # Display

ax = plt.gca()
ax.set_aspect('equal', adjustable='box')

plt.savefig('figure_auroc.png', dpi=1000)
plt.show()

In [ ]:
i = 385
def get_batch(X_train, index):
    i = index
    result = []
    for val in X_train:
        result.append(val[i])
    return np.array(result)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_2['EMBED'],
                                                   df_2['LABEL'],
                                                   test_size=0.2,
                                                   random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_test, 
                                               y_test, 
                                               test_size=0.5, 
                                               random_state=42)

X_train_i = get_batch(X_train, i)
clf_2 = LogisticRegression(
    max_iter=1000,
    n_jobs=-1,
    C=10
)


clf_2.fit(X_train_i, y_train.to_list())

X_test_i = get_batch(X_test, i)

test_pred_ALL = clf_2.predict_proba(X_test_i)

fpr, tpr, thresholds = metrics.roc_curve(y_test.to_list(), test_pred_ALL[:,1])
roc_auc = metrics.auc(fpr, tpr)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,
                                  estimator_name='Site 2')

X_train, X_test, y_train, y_test = train_test_split(df['EMBED'],
                                                   df['LABEL'],
                                                   test_size=0.2,
                                                   random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_test, 
                                               y_test, 
                                               test_size=0.5, 
                                               random_state=42)

X_train_i = get_batch(X_train, i)
clf = LogisticRegression(
    max_iter=1000,
    n_jobs=-1,
    C=10
)

clf.fit(X_train_i, y_train.to_list())

test_pred_ALL = clf.predict_proba(X_test_i)

fpr, tpr, thresholds = metrics.roc_curve(y_test.to_list(), test_pred_ALL[:,1])
roc_auc = metrics.auc(fpr, tpr)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,
                                  estimator_name='All sites')

display.plot()


plt.savefig('figure_lg_auroc.png', dpi=1000)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['EMBED'],
                                                   df['LABEL'],
                                                   test_size=0.2,
                                                   random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_test, 
                                               y_test, 
                                               test_size=0.5, 
                                               random_state=42)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import time

def get_batch(X_train, index):
    i = index
    result = []
    for val in X_train:
        result.append(val[i])
    return np.array(result)

results = []
for i in tqdm(range(500)):

    X_train_i = get_batch(X_train, i)
    clf = LogisticRegression(
        max_iter=1000,
        n_jobs=-1,
        C=10
    )

    clf.fit(X_train_i, y_train.to_list())

    X_test_i = get_batch(X_val, i)
    
    test_pred = clf.predict(X_test_i)
    results.append(
                    classification_report(
                          y_val.to_list(), 
                          test_pred,
                          output_dict=True)
        
    )


    print('------------------- Embedding {i} results ------------------'.format(i=i))
    print(results[-1]['0'])
    print(results[-1]['1'])

In [ ]:
def get_site(uniqueid):
    return uniqueid.split('.')[1]

X = []
for uniqueid in tqdm(df['UNIQUEID']):
    if not os.path.exists(mypath + 'site_' + get_site(uniqueid) + '/' + uniqueid + '.npy'):
        print('Oh no! ' + uniqueid + ' is missing!')
    else:
#         X.append(
#             np.load(mypath + 'site_' + get_site(uniqueid) + '/' + uniqueid + '.npy').flatten()
#         )
        
       X.append(
            np.load(mypath + 'site_' + get_site(uniqueid) + '/' + uniqueid + '.npy')
        )
        
X = pd.DataFrame({'EMBED':X})

y = [
    0 if label == 'S' else 1 for label in df['RIF_BINARY_PHENOTYPE']
]
y = pd.DataFrame({'LABEL':y})

df = pd.concat([df, X, y], axis=1)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['EMBED'],
                                                   df['LABEL'],
                                                   test_size=0.2,
                                                   random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_test, 
                                               y_test, 
                                               test_size=0.5, 
                                               random_state=42)

In [ ]:
df_ALL = df

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import time

def get_batch(X_train, index):
    i = index
    result = []
    for val in X_train:
        result.append(val[i])
    return np.array(result)

results = []
for i in tqdm(range(500)):

    X_train_i = get_batch(X_train, i)
    clf = LogisticRegression(
        max_iter=1000,
        n_jobs=-1,
        C=10
    )

    clf.fit(X_train_i, y_train.to_list())

    X_test_i = get_batch(X_val, i)
    
    test_pred = clf.predict(X_test_i)d
    results.append(
                    classification_report(
                          y_val.to_list(), 
                          test_pred,
                          output_dict=True)
        
    )


    print('------------------- Embedding {i} results ------------------'.format(i=i))
    print(results[-1]['0'])
    print(results[-1]['1'])

In [ ]:
def get_results(data, key, subkey):
    results = []
    for r in data:
        results.append(r[key][subkey])
    return results

In [ ]:
# df_ALL
results_ALL = results

In [ ]:
df_2 = df

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_2['EMBED'],
                                                   df_2['LABEL'],
                                                   test_size=0.2,
                                                   random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_test, 
                                               y_test, 
                                               test_size=0.5, 
                                               random_state=42)

i=385
def get_batch(X_train, index):
    i = index
    result = []
    for val in X_train:
        result.append(val[i])
    return np.array(result)

X_train_i = get_batch(X_train, i)
clf = LogisticRegression(
    max_iter=1000,
    n_jobs=-1,
    C=10
)

In [ ]:
i=385
def get_batch(X_train, index):
    i = index
    result = []
    for val in X_train:
        result.append(val[i])
    return np.array(result)

X_train_i = get_batch(X_train, i)
clf = LogisticRegression(
    max_iter=1000,
    n_jobs=-1,
    C=10
)

clf.fit(X_train_i, y_train.to_list())

X_test_i = get_batch(X_test, i)

test_pred_ALL = clf.predict_proba(X_test_i)

fpr, tpr, thresholds = metrics.roc_curve(y_test.to_list(), test_pred_ALL[:,1])
roc_auc = metrics.auc(fpr, tpr)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,
                                  estimator_name='All sites')
display.plot()
plt.savefig('figure_lg_auroc.png', dpi=1000)
plt.show()

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test.to_list(), test_pred_ALL[:,1])
roc_auc = metrics.auc(fpr, tpr)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,
                                  estimator_name='All sites')
display.plot()
plt.savefig('figure_lg_auroc.png', dpi=1000)
plt.show()

In [2]:
"""
    site sizes:
    {'02': 1062, '03': 1087, '04': 1486, 
     '05': 2576, '06': 1804, '08': 1274, 
     '10': 1584, '11': 354, '14': 336, 
     '17': 70, '20': 439}

"""
mypath = '/scratch/users/benson97/embeddings_rif/'
# resultspath = mypath + 'results/' + 'rif_predictions_200_400_full_embedding.npy'
df = pd.read_csv('/scratch/users/benson97/' + 'rif_labels.csv')

# modify the number of sites as 
# we upload more sites to Sherlock...
sites = [
    '02',
]

site_dfs = []
for s in sites:
    site_dfs.append(
        df[df.UNIQUEID.str.contains('site.' + s)].reset_index(drop=True)
    )

df = pd.concat(site_dfs).reset_index(drop=True)

def get_site(uniqueid):
    return uniqueid.split('.')[1]

X = []
for uniqueid in tqdm(df['UNIQUEID']):
    if not os.path.exists(mypath + 'site_' + get_site(uniqueid) + '/' + uniqueid + '.npy'):
        print('Oh no! ' + uniqueid + ' is missing!')
    else:
#         X.append(
#             np.load(mypath + 'site_' + get_site(uniqueid) + '/' + uniqueid + '.npy').flatten()
#         )
        
       X.append(
            np.load(mypath + 'site_' + get_site(uniqueid) + '/' + uniqueid + '.npy')
        )
        
X = pd.DataFrame({'EMBED':X})

y = [
    0 if label == 'S' else 1 for label in df['RIF_BINARY_PHENOTYPE']
]
y = pd.DataFrame({'LABEL':y})

df = pd.concat([df, X, y], axis=1)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['EMBED'],
                                                   df['LABEL'],
                                                   test_size=0.2,
                                                   random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_test, 
                                               y_test, 
                                               test_size=0.5, 
                                               random_state=42)


100%|██████████| 1062/1062 [00:22<00:00, 46.27it/s]


In [ ]:
i = 123
def get_batch(X_train, index):
    i = index
    result = []
    for val in X_train:
        result.append(val[i])
    return np.array(result)

X_train_i = get_batch(X_train, i)
clf = LogisticRegression(
    max_iter=1000,
    n_jobs=-1,
    C=10
)

clf.fit(X_train_i, y_train.to_list())

X_test_i = get_batch(X_test, i)

test_pred_2 = clf.predict(X_test_i)

In [ ]:
from numpy import dot
from numpy.linalg import norm

cos_sim = dot(a, b)/(norm(a)*norm(b))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['EMBED'],
                                                   df['LABEL'],
                                                   test_size=0.2,
                                                   random_state=42)

In [ ]:
i = 123
def get_batch(X_train, index):
    i = index
    result = []
    for val in X_train:
        result.append(val[i])
    return np.array(result)

X_train_i = get_batch(X_train, i)
clf = LogisticRegression(
    max_iter=1000,
    n_jobs=-1,
    C=10
)

clf.fit(X_train_i, y_train.to_list())

X_test_i = get_batch(X_test, i)

test_pred = clf.predict(X_test_i)

In [ ]:
classification_report(
      y_test.to_list(), 
      test_pred,
      output_dict=True)

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test.to_list(), test_pred)
roc_auc = metrics.auc(fpr, tpr)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,
                                  estimator_name='Site 2')
display.plot()
plt.savefig('figure_site2_auroc.png', dpi=1000)
plt.show()

In [ ]:
probs = clf.predict_proba(X_test_i)[:, 1]

In [ ]:
roc_auc_score(
    y_test.to_list(),
    probs
)

In [4]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import time

def get_batch(X_train, index):
    i = index
    result = []
    for val in X_train:
        result.append(val[i])
    return np.array(result)

results = []
for i in tqdm(range(500)):
#     start = time.time()
    X_train_i = get_batch(X_train, i)
    clf = LogisticRegression(
        max_iter=1000,
        n_jobs=-1,
        C=10
    )

    clf.fit(X_train_i, y_train.to_list())

    X_test_i = get_batch(X_val, i)
    
    test_pred = clf.predict(X_test_i)
    results.append(
                    classification_report(
                          y_val.to_list(), 
                          test_pred,
                          output_dict=True)
        
    )


#     print('------------------- Embedding {i} results ------------------'.format(i=i))
#     print(results[-1]['0'])
#     print(results[-1]['1'])

100%|██████████| 500/500 [02:50<00:00,  2.93it/s]


In [6]:
np.save('report500_site2_only.npy', results)

In [7]:
def get_results(data, key, subkey):
    results = []
    for r in data:
        results.append(r[key][subkey])
    return results

In [15]:
results = np.load('report500.npy', allow_pickle=True)

In [16]:
i_spec = np.argmax(get_results(results, '0', 'recall'))
i_sens = np.argmax(get_results(results, '1', 'recall'))
i_general = np.argmax(get_results(results, 'macro avg', 'f1-score'))

In [21]:
i_sens

123

In [22]:
i_general

123

In [17]:
print(
    results[i_spec],
)

{'0': {'precision': 0.6778149386845039, 'recall': 0.8194070080862533, 'f1-score': 0.7419158023184869, 'support': 742.0}, '1': {'precision': 0.567741935483871, 'recall': 0.37849462365591396, 'f1-score': 0.4541935483870968, 'support': 465.0}, 'accuracy': 0.6495443247721624, 'macro avg': {'precision': 0.6227784370841875, 'recall': 0.5989508158710837, 'f1-score': 0.5980546753527918, 'support': 1207.0}, 'weighted avg': {'precision': 0.6354090178159916, 'recall': 0.6495443247721624, 'f1-score': 0.6310700292628975, 'support': 1207.0}}


In [18]:
print(
    results[i_sens]
)

{'0': {'precision': 0.7448275862068966, 'recall': 0.7277628032345014, 'f1-score': 0.7361963190184049, 'support': 742.0}, '1': {'precision': 0.5809128630705395, 'recall': 0.6021505376344086, 'f1-score': 0.5913410770855333, 'support': 465.0}, 'accuracy': 0.6793703396851698, 'macro avg': {'precision': 0.662870224638718, 'recall': 0.6649566704344549, 'f1-score': 0.663768698051969, 'support': 1207.0}, 'weighted avg': {'precision': 0.6816789977575131, 'recall': 0.6793703396851698, 'f1-score': 0.6803904470227253, 'support': 1207.0}}


In [19]:
print(results[i_general])

{'0': {'precision': 0.7448275862068966, 'recall': 0.7277628032345014, 'f1-score': 0.7361963190184049, 'support': 742.0}, '1': {'precision': 0.5809128630705395, 'recall': 0.6021505376344086, 'f1-score': 0.5913410770855333, 'support': 465.0}, 'accuracy': 0.6793703396851698, 'macro avg': {'precision': 0.662870224638718, 'recall': 0.6649566704344549, 'f1-score': 0.663768698051969, 'support': 1207.0}, 'weighted avg': {'precision': 0.6816789977575131, 'recall': 0.6793703396851698, 'f1-score': 0.6803904470227253, 'support': 1207.0}}


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import time

def get_batch(X_train, index):
    i = index
    result = []
    for val in X_train:
        result.append(val[i])
    return np.array(result)

results = []
for i in range(500):
    start = time.time()
    X_train_i = get_batch(X_train, i)
    clf = LogisticRegression(
        max_iter=1000,
        n_jobs=-1,
        C=10
    )

    clf.fit(X_train_i, y_train.to_list())

    X_test_i = get_batch(X_val, i)
    
    test_pred = clf.predict(X_test_i)
    results.append(
                    classification_report(
                          y_val.to_list(), 
                          test_pred,
                          output_dict=True)
        
    )
    
    end = time.time()
    print('Iteration i required {s} seconds.'.format(s=(end - start)))


    print('------------------- Embedding {i} results ------------------'.format(i=i))
    print(results[-1]['0'])
    print(results[-1]['1'])


np.save(resultspath, results)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(
    n_estimators=100, 
    learning_rate=1.0,
    random_state=0)

clf.fit(X_train.to_list(), y_train.to_list())


test_pred = clf.predict(X_test.to_list())
results.append(
                classification_report(
                      y_test.to_list(), 
                      test_pred,
                      output_dict=True)
)

print('------------------- Embedding results ------------------')
print(results[-1]['0'])
print(results[-1]['1'])

np.save(resultspath, results)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier


results = []
for i in range(100):
    X_train_i = get_batch(X_train, i)
    clf = GradientBoostingClassifier(
        n_estimators=100, 
        learning_rate=1.0,
        random_state=0)
    
    clf.fit(X_train_i, y_train.to_list())

    
    X_test_i = get_batch(X_test, i)
    
    test_pred = clf.predict(X_test_i)
    results.append(
                    classification_report(
                          y_test.to_list(), 
                          test_pred,
                          output_dict=True)
    )
    
    print('------------------- Embedding {i} results ------------------'.format(i=i))
    print(results[-1]['0'])
    print(results[-1]['1'])


np.save(resultspath, results)

In [ ]:
# for the flattened version
# of the embeddings
clf = LogisticRegression(
    max_iter=2000,
    verbose=1

)

clf.fit(X_train.to_list(), y_train.to_list())

test_pred = clf.predict(X_test.to_list())
results.append(
                classification_report(
                      y_test.to_list(), 
                      test_pred,
                      output_dict=True)
)


print('---------------------- Embedding results ----------------------')
print(results[-1]['0'])
print(results[-1]['1'])

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

results = []
for i in range(100):
#     X_train_i = get_batch(X_train, i)
    clf = LogisticRegression(
        max_iter=500,
        verbose=51
                            
    )
    
    clf.fit(X_train.to_list(), y_train.to_list())

    
#     X_test_i = get_batch(X_test, i)
    
    test_pred = clf.predict(X_test.to_list())
    results.append(
                    classification_report(
                          y_test.to_list(), 
                          test_pred,
                          output_dict=True)
    )


    print('------------------- Embedding {i} results ------------------'.format(i=i))
    print(results[-1]['0'])
    print(results[-1]['1'])


np.save(resultspath, results)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier

results = []
for i in range(100):
    X_train_i = get_batch(X_train, i)
    clf = GradientBoostingClassifier(
        n_estimators=100, 
        learning_rate=1.0,
        random_state=0)
    
    clf.fit(X_train_i, y_train.to_list())

    
    X_test_i = get_batch(X_test, i)
    
    test_pred = clf.predict(X_test_i)
    results.append(
                    classification_report(
                          y_test.to_list(), 
                          test_pred,
                          output_dict=True)
    )
    
    print('------------------- Embedding {i} results ------------------'.format(i=i))
    print(results[-1]['0'])
    print(results[-1]['1'])


np.save(resultspath, results)

In [ ]:
# clf = LogisticRegression(random_state=42,
#                          max_iter=500,
#                          penalty=None
#                         )

# clf.fit(X_train.to_list(), y_train.to_list())

# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import classification_report

# train_pred = clf.predict(X_train.to_list())
# test_pred = clf.predict(X_test.to_list())
# print(
#     confusion_matrix(y_train.to_list(), train_pred),
#     confusion_matrix(y_test.to_list(), test_pred),
#     classification_report(y_test.to_list(), test_pred)
# )